<a href="https://colab.research.google.com/github/Nikhilsai-123/dailyassesment/blob/main/pspyk6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Json Data handling

In [ ]:
#Download sample file data
https://kloudkraft-dumps.s3.us-west-1.amazonaws.com/json/books1.json
https://kloudkraft-dumps.s3.us-west-1.amazonaws.com/json/colors.json

In [30]:
#Import Sparksesion
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd


In [8]:
#Create SparkSession
spark=SparkSession.builder.appName("nikhil").getOrCreate()

In [32]:
#Print dataframe schem
df=spark.read.option('header',True).json('/content/nikhil.parquet')
df1=spark.read.option('header',True).json('/content/nikhil2.json')


In [19]:
#Create dataframe from parquet file
%%writefile nikhil.json
{"id":"978-0641723445","cat":"book","name":"The Lightning Thief","author":"Rick Riordan","series_t":"Percy Jackson and the Olympians","sequence_i":1,"genre_s":"fantasy","inStock":true,"price":12.50,"pages_i":384}
{"id":"978-1423103349","cat":"paperback","name":"The Sea of Monsters","author":"Rick Riordan","series_t":"Percy Jackson and the Olympians","sequence_i":2,"genre_s":"fantasy","inStock":true,"price":6.49,"pages_i":304}
{"id":"978-1857995879","cat":"paperback","name":"Sophie's World : The Greek Philosophers","author":"Jostein Gaarder","sequence_i":1,"genre_s":"fantasy","inStock":true,"price":3.07,"pages_i":64}
{"id":"978-1923988177","cat":"paperback","name":"Lucene in Action, Second Edition","author":"Michael McCandless","sequence_i":1,"genre_s":"IT","inStock":true,"price":30.50,"pages_i":475}
{"id":"978-1933988077","cat":"paperback","name":"Lucene in Action, Second Edition","author":"Michael McCandless","sequence_i":1,"genre_s":"IT","inStock":true,"price":30.50,"pages_i":475}
{"id":"968-1933988177","cat":"paperback","name":"Lucene in Action, Second Edition","author":"Michael McCandless","sequence_i":1,"genre_s":"IT","inStock":true,"price":30.50,"pages_i":300}
{"id":"978-2933988077","cat":"paperback","name":"Lucene in Action, Second Edition","author":"Michael McCandless","sequence_i":1,"genre_s":"IT","inStock":true,"price":30.50,"pages_i":475}

Writing nikhil.json


In [22]:
#Show dataframe
%%writefile nikhil2.json
{"color": "black",  "category": "hue",  "type": "primary",  "code": {    "rgba": [255,255,255,1],    "hex": "#000"  },"other":{"type":"good"}}
{"color": "white","category": "value","code": {"rgba": [0,0,0,1],"hex": "#FFF"},"other":{"type":"good"}}
{"color": "red","category": "hue","type": "primary","code": {"rgba": [255,0,0,1],"hex": "#FF0"},"other":{"type":"ok"}}
{"color": "blue","category": "hue","type": "primary","code": {"rgba": [0,0,255,1],"hex": "#00F"},"other":{"type":"good"}}
{"color": "yellow","category": "hue","type": "primary","code": {"rgba": [255,255,0,1],"hex": "#FF0"},"other":{"type":"ok"}}
{"color": "green","category": "hue","type": "secondary","code": {"rgba": [0,255,0,1],"hex": "#0F0"},"other":{"type":"good"}}

Writing nikhil2.json


* Perform any 3 data cleaning operations

In [34]:
#print schema
df.printSchema()
df1.printSchema()

root
 |-- author: string (nullable = true)
 |-- cat: string (nullable = true)
 |-- genre_s: string (nullable = true)
 |-- id: string (nullable = true)
 |-- inStock: boolean (nullable = true)
 |-- name: string (nullable = true)
 |-- pages_i: long (nullable = true)
 |-- price: double (nullable = true)
 |-- sequence_i: long (nullable = true)
 |-- series_t: string (nullable = true)

root
 |-- category: string (nullable = true)
 |-- code: struct (nullable = true)
 |    |-- hex: string (nullable = true)
 |    |-- rgba: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |-- color: string (nullable = true)
 |-- other: struct (nullable = true)
 |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)



In [35]:
df.show()

+------------------+---------+-------+--------------+-------+--------------------+-------+-----+----------+--------------------+
|            author|      cat|genre_s|            id|inStock|                name|pages_i|price|sequence_i|            series_t|
+------------------+---------+-------+--------------+-------+--------------------+-------+-----+----------+--------------------+
|      Rick Riordan|     book|fantasy|978-0641723445|   true| The Lightning Thief|    384| 12.5|         1|Percy Jackson and...|
|      Rick Riordan|paperback|fantasy|978-1423103349|   true| The Sea of Monsters|    304| 6.49|         2|Percy Jackson and...|
|   Jostein Gaarder|paperback|fantasy|978-1857995879|   true|Sophie's World : ...|     64| 3.07|         1|                NULL|
|Michael McCandless|paperback|     IT|978-1923988177|   true|Lucene in Action,...|    475| 30.5|         1|                NULL|
|Michael McCandless|paperback|     IT|978-1933988077|   true|Lucene in Action,...|    475| 30.5| 

In [49]:
#1 sort top three desc
df.sort('pages_i',ascending=False).show(3)


+------------------+---------+-------+--------------+-------+--------------------+-------+-----+----------+--------+
|            author|      cat|genre_s|            id|inStock|                name|pages_i|price|sequence_i|series_t|
+------------------+---------+-------+--------------+-------+--------------------+-------+-----+----------+--------+
|Michael McCandless|paperback|     IT|978-1923988177|   true|Lucene in Action,...|    475| 30.5|         1|    NULL|
|Michael McCandless|paperback|     IT|978-1933988077|   true|Lucene in Action,...|    475| 30.5|         1|    NULL|
|Michael McCandless|paperback|     IT|978-2933988077|   true|Lucene in Action,...|    475| 30.5|         1|    NULL|
+------------------+---------+-------+--------------+-------+--------------------+-------+-----+----------+--------+
only showing top 3 rows



In [55]:
#O2 total pages for each book and author in desc
x=df.groupby('author').agg({'pages_i':'sum'})
x.sort('sum(pages_i)',ascending=False).show()

+------------------+------------+
|            author|sum(pages_i)|
+------------------+------------+
|Michael McCandless|        1725|
|      Rick Riordan|         688|
|   Jostein Gaarder|          64|
+------------------+------------+



In [50]:
#O3


df.fillna({'series_t':'0'}).show()

+------------------+---------+-------+--------------+-------+--------------------+-------+-----+----------+--------------------+
|            author|      cat|genre_s|            id|inStock|                name|pages_i|price|sequence_i|            series_t|
+------------------+---------+-------+--------------+-------+--------------------+-------+-----+----------+--------------------+
|      Rick Riordan|     book|fantasy|978-0641723445|   true| The Lightning Thief|    384| 12.5|         1|Percy Jackson and...|
|      Rick Riordan|paperback|fantasy|978-1423103349|   true| The Sea of Monsters|    304| 6.49|         2|Percy Jackson and...|
|   Jostein Gaarder|paperback|fantasy|978-1857995879|   true|Sophie's World : ...|     64| 3.07|         1|                   0|
|Michael McCandless|paperback|     IT|978-1923988177|   true|Lucene in Action,...|    475| 30.5|         1|                   0|
|Michael McCandless|paperback|     IT|978-1933988077|   true|Lucene in Action,...|    475| 30.5| 

In [62]:
df1.show()

+--------+--------------------+------+------+---------+
|category|                code| color| other|     type|
+--------+--------------------+------+------+---------+
|     hue|{#000, [255, 255,...| black|{good}|  primary|
|   value|{#FFF, [0, 0, 0, 1]}| white|{good}|     NULL|
|     hue|{#FF0, [255, 0, 0...|   red|  {ok}|  primary|
|     hue|{#00F, [0, 0, 255...|  blue|{good}|  primary|
|     hue|{#FF0, [255, 255,...|yellow|  {ok}|  primary|
|     hue|{#0F0, [0, 255, 0...| green|{good}|secondary|
+--------+--------------------+------+------+---------+



* Fetch any 5 insights from data

In [57]:
#Transformation1 avg price of each genre
df.groupBy('genre_s').agg({'price':'mean'}).show()

+-------+-----------------+
|genre_s|       avg(price)|
+-------+-----------------+
|fantasy|7.353333333333334|
|     IT|             30.5|
+-------+-----------------+



In [61]:
#Transformation2

df.filter(df.inStock==True).groupBy('genre_s').count().show()

+-------+-----+
|genre_s|count|
+-------+-----+
|fantasy|    3|
|     IT|    4|
+-------+-----+



In [65]:
#Transformation3

df1.groupBy('category').count().orderBy('count',ascending=False).show(1)

+--------+-----+
|category|count|
+--------+-----+
|     hue|    5|
+--------+-----+
only showing top 1 row



In [68]:
#Transformation4  counting number of primary

df1.filter(df1.type=='primary').count()


4

In [ ]:
#Transformation5


